In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [44]:
import pandas
import json
from os import path, listdir
import itertools
from collections import Counter
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from unidiff import PatchSet, PatchedFile
import sys
import numpy as np
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from commitgen.code import tokenize_code
from commitgen.nlp import tokenize_nlp
from commitgen.settings import data_path

In [40]:
def get_added_lines(parsed_diff):
    added_lines = []
    for modfile in parsed_diff.modified_files:
        for hunk in modfile:
            added_lines += [line.value for line in hunk if line.is_added]
    for addfile in parsed_diff.added_files:
        for hunk in addfile:
            added_lines += [line.value for line in hunk if line.is_added]
    return added_lines

def get_removed_lines(parsed_diff):
    removed_lines = []
    for modfile in parsed_diff.modified_files:
        for hunk in modfile:
            removed_lines += [line.value for line in hunk if line.is_removed]
    for remfile in parsed_diff.removed_files:
        for hunk in remfile:
            removed_lines += [line.value for line in hunk if line.is_removed]
    return removed_lines

In [41]:
json_path = path.join(data_path, "json")
diffs_path = path.join(data_path, "diffs")

diff_files = listdir(diffs_path)
json_files = listdir(json_path)

data_dict = {}
for filename in json_files:
    sha = filename.replace('.json','')
    filepath = path.join(json_path, filename)
    with open(filepath, 'r') as json_file:
        json_data = json.load(json_file)
    data_dict[sha] = json_data

diff_dict = {}
for filename in diff_files:
    sha = filename.replace('.diff','')
    filepath = path.join(diffs_path, filename)
    with open(filepath, 'r') as diff_file:
        diff = diff_file.read().decode('utf-8')
    parsed_diff = PatchSet(diff.splitlines())
    diff_dict[sha] = parsed_diff

In [42]:
# initial filter 
tokenized_comments = {}
for sha, commit in data_dict.items():
    t = tokenize_nlp(commit['commit']['message'])
    if len(t) > 1 and len(t) <= 30 :
        tokenized_comments[sha] = tokenize_nlp(commit['commit']['message'])

print len(tokenized_comments)

22426


In [43]:
filtered = [] 
for sha, comment in tokenized_comments.items():
    parsed_diff = diff_dict[sha] 
    added_lines   = get_added_lines(parsed_diff)
    removed_lines = get_removed_lines(parsed_diff)
    code = (added_lines , removed_lines)
    filtered.append((sha,comment, code))

In [45]:
pickle.dump( filtered, open( "filtered_original.pickle", "wb" ) )

In [54]:
# ahora , descartemos los cambios que tienen mas de un archivo (generamos un dataset atomico, atoumico) 

a = diff_dict.items()

atomic_diff = {}
for aa in a:
    if len(aa[1]) > 1:
        atomic_diff[aa[0]] = aa[1]

In [55]:
len(atomic_diff)

8290

In [56]:
atomic_diff_keys = atomic_diff.keys()

In [60]:
# checking *
def intersect(a, b):
    return list(set(a) & set(b))

valid_keys = intersect(atomic_diff_keys, tokenized_comments.keys() )

In [61]:
len(valid_keys)

7458

In [62]:
filtered_atomic = [] 
for k in valid_keys:
    parsed_diff = atomic_diff[k] 
    added_lines   = get_added_lines(parsed_diff)
    removed_lines = get_removed_lines(parsed_diff)
    code = (added_lines , removed_lines)
    
    comment = tokenized_comments[k]
    filtered_atomic.append((k,comment, code))
    

In [64]:
pickle.dump( filtered_atomic, open( "filtered_atomic.pickle", "wb" ) )